In [ ]:
import requests
from datetime import datetime, timedelta
import pytz

# Airtable setup
API_KEY = "your_airtable_token_here"
BASE_ID = "app59RhWQEcM6gMLt"
TABLE_NAME = "Imported table"
URL = f"https://api.airtable.com/v0/{BASE_ID}/{TABLE_NAME}"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

# Only update if last check was over 30 days ago
REFRESH_THRESHOLD = timedelta(days=30)

# Function to determine if update is needed
def should_refresh(timestamp_str):
    if not timestamp_str:
        return True
    timestamp = datetime.fromisoformat(timestamp_str.replace("Z", "+00:00"))
    return datetime.now(pytz.utc) - timestamp > REFRESH_THRESHOLD

# Get records from Airtable
def get_records():
    records = []
    offset = None
    while True:
        params = {"pageSize": 100}
        if offset:
            params["offset"] = offset
        response = requests.get(URL, headers=HEADERS, params=params).json()
        records.extend(response.get("records", []))
        offset = response.get("offset")
        if not offset:
            break
    return records

# Update Last Parsed Time field
def update_record(record_id):
    now_utc = datetime.now(pytz.utc).isoformat()
    update_payload = {
        "records": [
            {
                "id": record_id,
                "fields": {
                    "Last Parsed Time": now_utc
                }
            }
        ]
    }
    response = requests.patch(URL, headers=HEADERS, json=update_payload)
    print("Airtable response:", response.status_code, response.text)

# Main loop
def main():
    print("Scanning Airtable records...")
    records = get_records()

    for record in records:
        fields = record.get("fields", {})
        name = fields.get("FullName", "Unknown")
        linkedin_url = fields.get("LinkedInUrl")
        last_parsed = fields.get("Last Parsed Time")

        if linkedin_url and should_refresh(last_parsed):
            print(f"Updating: {name} — {linkedin_url}")
            update_record(record["id"])
        else:
            print(f"Skipped: {name} — up to date.")

    print("Process complete.")

# Run it!
if __name__ == "__main__":
    main()


🔎 Scanning Airtable records...
🔁 Updating: Julian Lach — https://www.linkedin.com/in/julian-lach-062400150/
📡 Airtable response: 200 {"records":[{"id":"rec01vFVTpSJzLuXm","createdTime":"2025-03-25T23:25:46.000Z","fields":{"FullName":"Julian Lach","FieldOfStudy":"Applied Information Management Systems","GraduationYear2":2021,"JobTitle":"Technology Consultant @ EY","LinkedInUrl":"https://www.linkedin.com/in/julian-lach-062400150/","Rating":"Great","Last Parsed Time":"2025-04-10T17:04:46.075Z"}}]}
🔁 Updating: Anshu Arora — https://www.linkedin.com/in/anshu-arora-4393822/
📡 Airtable response: 200 {"records":[{"id":"rec06mTVFx0w9kNdV","createdTime":"2025-03-25T23:25:46.000Z","fields":{"FullName":"Anshu Arora","FieldOfStudy":"Management Information Systems","GraduationYear2":1998,"JobTitle":"Senior Director, Corporate Security @Verizon","LinkedInUrl":"https://www.linkedin.com/in/anshu-arora-4393822/","Last Parsed Time":"2025-04-10T17:04:46.346Z"}}]}
🔁 Updating: Max Kaiser — https://www.linke